### **Aprendizaje por refuerzo y toma de decisiones**

Este cuaderno integra y reorganiza código para cubrir los siguientes temas:

- **Marco MDP:** estados, acciones, recompensas, políticas y valores.  
- **Métodos básicos:** *Q-learning* y *Policy Gradient* (visión general).  
- **RL en** juegos, robótica (control), y sistemas de recomendación (bandits).  
- **Conexión RL-LLM:** RLHF como RL sobre recompensas humanas (pipeline simplificado).  
- **Diseño de recompensas** y desafíos de **estabilidad/seguridad**.

> Algunas secciones están marcadas como opcionales para mantener el tiempo de ejecución razonable.


#### **0. Preparación del entorno**

Este cuaderno usa **PyTorch** y, si está disponible, un entorno tipo **Gym/Gymnasium** para ejemplos de control.  
Si tu entorno no tiene `gymnasium`/`gym`, puedes ejecutar sin problemas las partes de *GridWorld* y *Bandits*.


In [ ]:
import math
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

# Compatibilidad Gym vs Gymnasium (si existe)
try:
    import gymnasium as gym
    _GYMNASIUM = True
except Exception:
    try:
        import gym
        _GYMNASIUM = False
    except Exception:
        gym = None
        _GYMNASIUM = False

def reset_env(env, seed=None):
    """Devuelve obs de forma compatible con gymnasium/gym."""
    if seed is not None:
        try:
            out = env.reset(seed=seed)
        except TypeError:
            out = env.reset()
    else:
        out = env.reset()
    if isinstance(out, tuple):
        return out[0]
    return out

def step_env(env, action):
    """Devuelve (obs, reward, done) compatible con gymnasium/gym."""
    out = env.step(action)
    if len(out) == 5:  # gymnasium: obs, reward, terminated, truncated, info
        obs, reward, terminated, truncated, _ = out
        done = terminated or truncated
        return obs, reward, done
    else:  # gym: obs, reward, done, info
        obs, reward, done, _ = out
        return obs, reward, done

# Reproducibilidad mínima
SEED = 7
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


#### **1. Marco MDP: definición operacional (con un GridWorld mínimo)**

Un **MDP** se define por $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$:

- **Estado** $s \in \mathcal{S}$: lo que el agente observa del mundo.
- **Acción** $a \in \mathcal{A}$: decisión del agente.
- **Transición** $P(s' \mid s, a)$: dinámica del entorno.
- **Recompensa** $R(s,a,s')$: señal de utilidad.
- **Descuento** $\gamma \in [0,1)$: preferencia por recompensas futuras.

Implementamos un GridWorld pequeño para tener control total (sin dependencias externas).


In [ ]:
from dataclasses import dataclass

@dataclass
class GridWorld:
    '''
    GridWorld 4x4:
      - Inicio: (0,0)
      - Meta:   (3,3) recompensa +1 y termina
      - Pozo:   (1,1) recompensa -1 y termina (opcional)
      - Penalización por paso: -0.01 (para evitar loops)
    '''
    n: int = 4
    goal: tuple = (3, 3)
    pit: tuple = (1, 1)
    step_penalty: float = -0.01
    gamma: float = 0.99

    def __post_init__(self):
        self.state = (0, 0)

    @property
    def state_dim(self):
        return self.n * self.n

    @property
    def action_dim(self):
        return 4  # 0=up,1=right,2=down,3=left

    def reset(self):
        self.state = (0, 0)
        return self._encode(self.state)

    def _encode(self, s):
        # one-hot
        x = np.zeros(self.state_dim, dtype=np.float32)
        idx = s[0] * self.n + s[1]
        x[idx] = 1.0
        return x

    def step(self, a):
        r, c = self.state
        if a == 0:   r = max(0, r - 1)
        elif a == 1: c = min(self.n - 1, c + 1)
        elif a == 2: r = min(self.n - 1, r + 1)
        elif a == 3: c = max(0, c - 1)

        next_state = (r, c)

        reward = self.step_penalty
        done = False
        if next_state == self.goal:
            reward = 1.0
            done = True
        elif next_state == self.pit:
            reward = -1.0
            done = True

        self.state = next_state
        return self._encode(next_state), reward, done

env_gw = GridWorld()
obs = env_gw.reset()
obs.shape, env_gw.action_dim


##### **1.1 Política y valor: evaluación Monte Carlo (MC)**

Una **política** $\pi(a\mid s)$ define cómo se eligen acciones.  
El **retorno descontado** es $G = \sum_{t=0}^{T-1} \gamma^t r_t$.  
Aproximamos $V^\pi(s)$ como promedio de retornos desde $s$.


In [ ]:
def run_episode_gw(env: GridWorld, policy_fn, max_steps=100):
    s = env.reset()
    total = 0.0
    g = 1.0
    for _ in range(max_steps):
        a = policy_fn(s)
        s, r, done = env.step(a)
        total += g * r
        g *= env.gamma
        if done:
            break
    return total

def random_policy(_s):
    return random.randrange(env_gw.action_dim)

vals = [run_episode_gw(env_gw, random_policy) for _ in range(2000)]
float(np.mean(vals)), float(np.std(vals))


#### **2. Método básico 1: Q-learning (tabular)**

Aprendemos una función $Q(s,a)$ y la actualizamos con:

$$
Q(s,a)\leftarrow Q(s,a) + \alpha\Big(r + \gamma \max_{a'}Q(s',a') - Q(s,a)\Big)
$$

Usaremos **epsilon-greedy** para explorar.


In [ ]:
def onehot_to_index(onehot):
    return int(np.argmax(onehot))

def q_learning_gridworld(env: GridWorld, episodes=2000, alpha=0.1, gamma=0.99, eps_start=1.0, eps_end=0.05):
    Q = np.zeros((env.state_dim, env.action_dim), dtype=np.float32)
    rewards = []
    for ep in range(episodes):
        s = env.reset()
        s_idx = onehot_to_index(s)
        eps = eps_end + (eps_start - eps_end) * math.exp(-ep / (episodes/5))
        total = 0.0

        for _ in range(200):
            if random.random() < eps:
                a = random.randrange(env.action_dim)
            else:
                a = int(np.argmax(Q[s_idx]))

            s2, r, done = env.step(a)
            s2_idx = onehot_to_index(s2)

            td_target = r + gamma * float(np.max(Q[s2_idx])) * (0.0 if done else 1.0)
            Q[s_idx, a] += alpha * (td_target - float(Q[s_idx, a]))

            total += r
            s_idx = s2_idx
            if done:
                break

        rewards.append(total)
    return Q, rewards

Q_tab, ep_rewards = q_learning_gridworld(env_gw, episodes=3000)
np.mean(ep_rewards[-200:]), np.mean(ep_rewards[:200])


In [ ]:
def greedy_policy_from_Q(Q):
    def policy(s_onehot):
        s_idx = onehot_to_index(s_onehot)
        return int(np.argmax(Q[s_idx]))
    return policy

pi_greedy = greedy_policy_from_Q(Q_tab)

def rollout_and_print(env, policy, max_steps=30):
    s = env.reset()
    def idx_to_rc(idx):
        return (idx // env.n, idx % env.n)
    traj = []
    for _ in range(max_steps):
        idx = onehot_to_index(s)
        traj.append(idx_to_rc(idx))
        a = policy(s)
        s, _, done = env.step(a)
        if done:
            idx = onehot_to_index(s)
            traj.append(idx_to_rc(idx))
            break
    return traj

rollout_and_print(env_gw, pi_greedy)


#### **3. Extensión: DQN (Deep Q-Network)**

Cuando el estado es grande/continuo, se aproxima $Q(s,a)$ con una red (**DQN**).


In [ ]:
#DQN (Deep Q-Network)
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def update_dqn(policy_network, target_network, optimizer, replay_buffer, batch_size=64, gamma=0.99):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)

    q_values = policy_network(state).gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_values = target_network(next_state).max(1)[0]
    expected_q_values = reward + gamma * next_q_values * (1 - done)

    loss = torch.nn.functional.mse_loss(q_values, expected_q_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# Complemento para que el bloque DQN sea ejecutable (ReplayBuffer + loop corto) 

from collections import deque

class ReplayBuffer:
    def __init__(self, capacity=50_000):
        self.buffer = deque(maxlen=capacity)

    def add(self, s, a, r, s2, done):
        self.buffer.append((s, a, r, s2, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        s, a, r, s2, d = zip(*batch)
        state      = torch.tensor(np.array(s),  dtype=torch.float32)
        action     = torch.tensor(np.array(a),  dtype=torch.int64)
        reward     = torch.tensor(np.array(r),  dtype=torch.float32)
        next_state = torch.tensor(np.array(s2), dtype=torch.float32)
        done       = torch.tensor(np.array(d),  dtype=torch.float32)
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)

def run_dqn_cartpole_short(steps=5000, batch_size=64, target_sync=500):
    if gym is None:
        raise RuntimeError("No se encontró gym/gymnasium.")

    env = gym.make("CartPole-v1")
    s = reset_env(env, seed=SEED)
    state_dim = len(s)
    action_dim = env.action_space.n

    policy_net = DQN(state_dim, action_dim)
    target_net = DQN(state_dim, action_dim)
    target_net.load_state_dict(policy_net.state_dict())

    opt = optim.Adam(policy_net.parameters(), lr=1e-3)
    rb = ReplayBuffer(capacity=50_000)

    eps = 1.0
    eps_min = 0.05
    eps_decay = 0.995

    s = reset_env(env)
    ep_return = 0.0
    returns = []

    for t in range(1, steps + 1):
        # epsilon-greedy
        if random.random() < eps:
            a = env.action_space.sample()
        else:
            with torch.no_grad():
                q = policy_net(torch.tensor(s, dtype=torch.float32).unsqueeze(0))
                a = int(torch.argmax(q, dim=1).item())

        s2, r, done = step_env(env, a)
        rb.add(s, a, r, s2, float(done))
        s = s2
        ep_return += float(r)

        if done:
            returns.append(ep_return)
            s = reset_env(env)
            ep_return = 0.0
            eps = max(eps_min, eps * eps_decay)

        if len(rb) >= batch_size:
            update_dqn(policy_net, target_net, opt, rb, batch_size=batch_size)

        if t % target_sync == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if t % 1000 == 0 and len(returns) > 0:
            print(f"step {t:5d} | eps={eps:.2f} | retorno promedio (últimos 10)={np.mean(returns[-10:]):.1f}")

    env.close()
    return returns

# Ejecuta si quieres una demostración rápida:
# dqn_returns = run_dqn_cartpole_short(steps=6000)


> Ejercicio: ¿dónde añadirías **experience replay** y una **red objetivo** (*target network*) para estabilizar DQN?


In [ ]:
## Tus respuestas

#### **4. Policy Gradient (REINFORCE)**

En lugar de aprender $Q$, parametrizamos directamente $\pi_\theta(a\mid s)$.  
REINFORCE usa $\nabla_\theta \log \pi_\theta(a_t\mid s_t)\,G_t$.



In [ ]:
# Policy Gradient
import torch
import torch.nn as nn
import torch.optim as optim

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

def update_policy(policy_network, optimizer, rewards, log_probs):
    discounted_rewards = []
    cumulative_reward = 0
    for reward in rewards[::-1]:
        cumulative_reward = reward + 0.99 * cumulative_reward
        discounted_rewards.insert(0, cumulative_reward)

    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)

    policy_loss = []
    for log_prob, reward in zip(log_probs, discounted_rewards):
        policy_loss.append(-log_prob * reward)

    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()


In [ ]:
def reinforce_cartpole(num_episodes=200, lr=1e-2, gamma=0.99, max_steps=500):
    if gym is None:
        raise RuntimeError("No se encontró gym/gymnasium. Ejecuta esta sección en un entorno con gym instalado.")
    env = gym.make("CartPole-v1")

    s0 = reset_env(env, seed=SEED)
    state_dim = len(s0)
    action_dim = env.action_space.n

    policy = PolicyNetwork(state_dim, action_dim)
    optimizer = optim.Adam(policy.parameters(), lr=lr)

    history = []

    for ep in range(num_episodes):
        s = reset_env(env)
        rewards, log_probs = [], []
        ep_return = 0.0

        for _ in range(max_steps):
            s_t = torch.tensor(s, dtype=torch.float32).unsqueeze(0)
            probs = policy(s_t)
            dist = torch.distributions.Categorical(probs)
            a = dist.sample()
            log_probs.append(dist.log_prob(a))

            s, r, done = step_env(env, int(a.item()))
            rewards.append(float(r))
            ep_return += float(r)
            if done:
                break

        update_policy(policy, optimizer, rewards, log_probs)
        history.append(ep_return)

        if (ep + 1) % 25 == 0:
            print(f"EP {ep+1:4d} | retorno promedio (últimos 25): {np.mean(history[-25:]):.1f}")

    env.close()
    return history

# Ejecuta si tienes gym instalado:
# hist = reinforce_cartpole(num_episodes=200)


##### **4.1 Estabilidad (concepto): varianza y baseline**

REINFORCE suele tener alta varianza. Una mejora mínima es restar un **baseline** $b(s)$:

$$
\nabla_\theta \log \pi_\theta(a_t\mid s_t)\,(G_t - b(s_t))
$$

> Ejercicio: modifica `update_policy` para incluir un baseline constante o aprende un crítico $V_\phi(s)$.


#### **5. Aplicaciones: juegos, "robótica" (control), recomendación (bandits)**

- **Juegos (discretos):** CartPole/FrozenLake -> estados relativamente pequeños.
- **Control continuo (proxy de robótica):** Pendulum -> acciones continuas, suelen requerir actor-critic.
- **Recomendación:** frecuentemente **bandit contextual** (una decisión por interacción) o RL secuencial (slates).


In [ ]:
# (A) Juego / control discreto: CartPole (si gym existe)
if gym is not None:
    env_cp = gym.make("CartPole-v1")
    s = reset_env(env_cp, seed=SEED)
    print("CartPole: dim estado =", len(s), "| acciones =", env_cp.action_space.n)
    env_cp.close()
else:
    print("gym no disponible: salta ejemplos CartPole/Pendulum.")


In [ ]:
# (B) Proxy de robótica / control continuo: Pendulum (si gym existe)
if gym is not None:
    try:
        env_pd = gym.make("Pendulum-v1")
        s = reset_env(env_pd, seed=SEED)
        print("Pendulum: dim estado =", len(s), "| acción shape =", env_pd.action_space.shape)
        env_pd.close()
    except Exception as e:
        print("No se pudo crear Pendulum-v1:", e)


##### **5.1 Recomendación como Bandit Contextual (mini-entorno)**

Contexto $x$, acción $a$ (ítem), recompensa $r$ (click/tiempo/etc.).  
Comparamos **epsilon-greedy** con modelo lineal vs **LinUCB**.


In [ ]:
class ContextualBandit:
    def __init__(self, d=5, k=5, noise=0.1):
        self.d = d
        self.k = k
        self.noise = noise
        self.theta = np.random.randn(k, d).astype(np.float32)  # parámetro real por acción

    def sample_context(self):
        x = np.random.randn(self.d).astype(np.float32)
        x /= (np.linalg.norm(x) + 1e-8)
        return x

    def reward(self, x, a):
        mu = float(np.dot(self.theta[a], x))
        return mu + np.random.randn() * self.noise

def linucb(bandit, T=2000, alpha=1.0):
    d, k = bandit.d, bandit.k
    A = [np.eye(d, dtype=np.float32) for _ in range(k)]
    b = [np.zeros(d, dtype=np.float32) for _ in range(k)]
    rewards = []

    for _ in range(T):
        x = bandit.sample_context()
        p = []
        for a in range(k):
            A_inv = np.linalg.inv(A[a])
            theta_hat = A_inv @ b[a]
            mean = float(theta_hat @ x)
            conf = alpha * math.sqrt(float(x @ A_inv @ x))
            p.append(mean + conf)

        a = int(np.argmax(p))
        r = bandit.reward(x, a)
        A[a] += np.outer(x, x)
        b[a] += r * x
        rewards.append(r)

    return rewards

def eps_greedy_linear(bandit, T=2000, eps=0.1, lr=0.05):
    d, k = bandit.d, bandit.k
    W = np.zeros((k, d), dtype=np.float32)
    rewards = []

    for _ in range(T):
        x = bandit.sample_context()
        if random.random() < eps:
            a = random.randrange(k)
        else:
            a = int(np.argmax(W @ x))
        r = bandit.reward(x, a)
        err = r - float(W[a] @ x)
        W[a] += lr * err * x
        rewards.append(r)

    return rewards

bandit = ContextualBandit(d=6, k=6, noise=0.2)
r_eps = eps_greedy_linear(bandit, T=3000, eps=0.1, lr=0.05)
r_ucb = linucb(bandit, T=3000, alpha=1.0)

print("Promedio últimas 500 (eps-greedy):", np.mean(r_eps[-500:]))
print("Promedio últimas 500 (LinUCB):    ", np.mean(r_ucb[-500:]))


#### **6. Conexión RL–LLM: RLHF como RL sobre recompensas humanas (control -> intuición)**

RLHF típico:

1) **SFT** (imitación)  
2) **Reward Model** (preferencias)  
3) **RL (PPO) + restricción KL**  

Primero: RLHF "tipo control" (CartPole) con **preferencias simuladas**.


In [ ]:
def human_score_cartpole_state(s):
    # s = [x, x_dot, theta, theta_dot]
    x, xdot, theta, thetadot = s
    return - (abs(theta) + 0.1 * abs(x) + 0.01 * abs(thetadot))

def generate_state_pairs_from_env(env_name="CartPole-v1", num_pairs=600, rollout_len=50):
    if gym is None:
        raise RuntimeError("No se encontró gym/gymnasium.")
    env = gym.make(env_name)

    pairs_i, pairs_j, prefs = [], [], []

    for _ in range(num_pairs):
        s = reset_env(env)
        for _t in range(random.randrange(1, rollout_len)):
            a = env.action_space.sample()
            s, _, done = step_env(env, a)
            if done:
                s = reset_env(env)
        s_i = np.array(s, dtype=np.float32)

        s = reset_env(env)
        for _t in range(random.randrange(1, rollout_len)):
            a = env.action_space.sample()
            s, _, done = step_env(env, a)
            if done:
                s = reset_env(env)
        s_j = np.array(s, dtype=np.float32)

        pref = 1 if human_score_cartpole_state(s_i) > human_score_cartpole_state(s_j) else 0
        pairs_i.append(s_i); pairs_j.append(s_j); prefs.append(pref)

    env.close()
    state_i = torch.tensor(np.stack(pairs_i), dtype=torch.float32)
    state_j = torch.tensor(np.stack(pairs_j), dtype=torch.float32)
    preferences = torch.tensor(np.array(prefs), dtype=torch.float32)
    return (state_i, state_j), preferences


In [ ]:
class BradleyTerryRewardModel(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        return self.fc2(x)

def preference_probability(model, state_i, state_j):
    score_i = model(state_i)
    score_j = model(state_j)
    return torch.sigmoid(score_i - score_j)

def train_reward_model_bt(model, optimizer, states, preferences, num_epochs=200):
    loss_fn = nn.BCELoss()
    state_i, state_j = states
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        probs = preference_probability(model, state_i, state_j).squeeze()
        loss = loss_fn(probs, preferences)
        loss.backward()
        optimizer.step()
        if epoch % 50 == 0:
            with torch.no_grad():
                acc = ((probs > 0.5).float() == preferences).float().mean().item()
            print(f"Epoca {epoch:3d} | loss={loss.item():.4f} | acc={acc:.3f}")

if gym is not None:
    (si, sj), prefs = generate_state_pairs_from_env(num_pairs=800)
    rm = BradleyTerryRewardModel(state_dim=si.shape[1])
    opt_rm = optim.Adam(rm.parameters(), lr=1e-3)
    train_reward_model_bt(rm, opt_rm, (si, sj), prefs, num_epochs=200)


In [ ]:
class SmallPolicy(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return torch.softmax(self.fc2(x), dim=-1)

def collect_trajectory_scores(env, policy, rm, max_steps=200):
    s = reset_env(env)
    log_probs = []
    scores = []
    for _ in range(max_steps):
        st = torch.tensor(s, dtype=torch.float32).unsqueeze(0)
        probs = policy(st)
        dist = torch.distributions.Categorical(probs)
        a = dist.sample()
        log_probs.append(dist.log_prob(a))

        score = rm(st).squeeze()
        scores.append(score)

        s, _, done = step_env(env, int(a.item()))
        if done:
            break
    return log_probs, scores

def policy_gradient_update(policy, optimizer, log_probs, rewards_like, gamma=0.99):
    returns = []
    G = torch.tensor(0.0)
    for r in reversed(rewards_like):
        G = r + gamma * G
        returns.insert(0, G)
    returns = torch.stack(returns)
    returns = (returns - returns.mean()) / (returns.std() + 1e-8)

    loss = -(torch.stack(log_probs) * returns.detach()).sum()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return float(loss.item())

def train_policy_with_rm(num_episodes=80, lr=2e-3):
    if gym is None:
        raise RuntimeError("No se encontró gym/gymnasium.")
    env = gym.make("CartPole-v1")
    s0 = reset_env(env, seed=SEED)
    policy = SmallPolicy(len(s0), env.action_space.n)
    opt = optim.Adam(policy.parameters(), lr=lr)

    for ep in range(num_episodes):
        logp, scores = collect_trajectory_scores(env, policy, rm, max_steps=300)
        loss = policy_gradient_update(policy, opt, logp, scores)
        if (ep + 1) % 20 == 0:
            print(f"EP {ep+1:3d} | loss={loss:.3f} | steps={len(scores)}")

    env.close()
    return policy

# Ejecuta si RM se entrenó:
# policy_rm = train_policy_with_rm()


#### **7. RL y LLM (pipeline simplificado)**

Sin usar Transformers, reproducimos el **esqueleto RLHF**:

- Tokenización simple (vocabulario pequeño).
- Reward Model entrenado con preferencias.
- Policy optimization con penalización KL a la política base.


In [ ]:
VOCAB = ["A","B","C","D","E","<eos>"]
stoi = {t:i for i,t in enumerate(VOCAB)}
itos = {i:t for t,i in stoi.items()}

def decode(seq):
    return "".join(itos[i] for i in seq if itos[i] != "<eos>")

def sample_sequence(step_logits_fn, max_len=8):
    seq = []
    for t in range(max_len):
        logits = step_logits_fn(t)
        probs = torch.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs)
        tok = int(dist.sample().item())
        seq.append(tok)
        if itos[tok] == "<eos>":
            break
    return seq

def human_score_text(seq):
    s = decode(seq)
    return 2.0*s.count("A") - 0.2*len(s)

def generate_text_preferences(num_pairs=400, max_len=8):
    ref_logits = torch.zeros(max_len, len(VOCAB))
    pairs = []
    labels = []
    for _ in range(num_pairs):
        si = sample_sequence(lambda t: ref_logits[t], max_len=max_len)
        sj = sample_sequence(lambda t: ref_logits[t], max_len=max_len)
        yi = 1 if human_score_text(si) > human_score_text(sj) else 0
        pairs.append((si, sj))
        labels.append(yi)
    return pairs, torch.tensor(labels, dtype=torch.float32)

pairs, y = generate_text_preferences()
pairs[0], y[0].item(), decode(pairs[0][0]), decode(pairs[0][1])


In [ ]:
def featurize(seq):
    v = np.zeros(len(VOCAB), dtype=np.float32)
    for tok in seq:
        v[tok] += 1.0
    v /= (v.sum() + 1e-8)
    return v

X_i = torch.tensor(np.stack([featurize(si) for si, _ in pairs]), dtype=torch.float32)
X_j = torch.tensor(np.stack([featurize(sj) for _, sj in pairs]), dtype=torch.float32)

class SeqRewardModel(nn.Module):
    def __init__(self, V):
        super().__init__()
        self.fc1 = nn.Linear(V, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

def train_rm_seq(model, optimizer, X_i, X_j, y, epochs=300):
    bce = nn.BCELoss()
    for e in range(epochs):
        optimizer.zero_grad()
        si = model(X_i)
        sj = model(X_j)
        p = torch.sigmoid(si - sj).squeeze()
        loss = bce(p, y)
        loss.backward()
        optimizer.step()
        if e % 75 == 0:
            acc = ((p > 0.5).float() == y).float().mean().item()
            print(f"epoca {e:3d} | loss={loss.item():.4f} | acc={acc:.3f}")

rm_seq = SeqRewardModel(len(VOCAB))
opt_rm = optim.Adam(rm_seq.parameters(), lr=1e-3)
train_rm_seq(rm_seq, opt_rm, X_i, X_j, y, epochs=300)


In [ ]:
class ToySequencePolicy(nn.Module):
    def __init__(self, max_len, V):
        super().__init__()
        self.logits = nn.Parameter(torch.zeros(max_len, V))

    def step_logits(self, t):
        return self.logits[t]

def seq_logprob(policy: ToySequencePolicy, seq):
    lp = torch.tensor(0.0)
    for t, tok in enumerate(seq):
        probs = torch.softmax(policy.step_logits(t), dim=-1)
        lp = lp + torch.log(probs[tok] + 1e-12)
        if itos[tok] == "<eos>":
            break
    return lp

def kl_stepwise(policy, ref_logits):
    kls = []
    for t in range(ref_logits.shape[0]):
        p = torch.softmax(policy.step_logits(t), dim=-1)
        q = torch.softmax(ref_logits[t], dim=-1)
        kls.append((p * (torch.log(p + 1e-12) - torch.log(q + 1e-12))).sum())
    return torch.stack(kls).mean()

def rm_reward_for_seq(seq):
    x = torch.tensor(featurize(seq), dtype=torch.float32).unsqueeze(0)
    return rm_seq(x).squeeze()

def train_toy_rlhf(steps=400, batch=32, beta=0.1, lr=5e-2, max_len=8):
    ref_logits = torch.zeros(max_len, len(VOCAB))
    policy = ToySequencePolicy(max_len, len(VOCAB))
    opt = optim.SGD(policy.parameters(), lr=lr)

    for it in range(steps):
        opt.zero_grad()
        seqs = [sample_sequence(lambda t: policy.step_logits(t), max_len=max_len) for _ in range(batch)]
        rewards = torch.stack([rm_reward_for_seq(s) for s in seqs])
        logps = torch.stack([seq_logprob(policy, s) for s in seqs])

        pg_loss = -(rewards.detach() * logps).mean()
        kl = kl_stepwise(policy, ref_logits)
        loss = pg_loss + beta * kl

        loss.backward()
        opt.step()

        if (it + 1) % 80 == 0:
            ex = sample_sequence(lambda t: policy.step_logits(t), max_len=max_len)
            print(f"it {it+1:3d} | loss={loss.item():.3f} | rm_reward={rm_reward_for_seq(ex).item():.2f} | ejemplo='{decode(ex)}'")

    return policy

toy_policy = train_toy_rlhf()


#### **8. Diseño de recompensas y desafíos de estabilidad/seguridad**

Problemas típicos:

- **Reward hacking / Goodhart:** maximiza la señal, no el objetivo real.
- **Shaping mal calibrado:** induce políticas no deseadas.
- **Inestabilidad:** bootstrapping + aproximación + off-policy puede divergir.


In [ ]:
class LoopTrapEnv:
    '''
    Línea 0..4, acciones: 0=left, 1=right
    Objetivo: llegar a 4 (+1 y termina)
    Recompensa mal diseñada: +0.2 al visitar estado 2 (loop trap)
    '''
    def __init__(self):
        self.pos = 0
        self.done = False

    def reset(self):
        self.pos = 0
        self.done = False
        return self.pos

    def step(self, a):
        if self.done:
            return self.pos, 0.0, True
        if a == 0:
            self.pos = max(0, self.pos - 1)
        else:
            self.pos = min(4, self.pos + 1)

        r = 0.0
        if self.pos == 2:
            r += 0.2
        if self.pos == 4:
            r += 1.0
            self.done = True
        return self.pos, r, self.done

def qlearn_looptrap(episodes=2000, alpha=0.2, gamma=0.95, eps=0.2):
    env = LoopTrapEnv()
    Q = np.zeros((5, 2), dtype=np.float32)
    for _ in range(episodes):
        s = env.reset()
        for _t in range(50):
            a = random.randrange(2) if random.random() < eps else int(np.argmax(Q[s]))
            s2, r, done = env.step(a)
            target = r + gamma * float(np.max(Q[s2])) * (0.0 if done else 1.0)
            Q[s, a] += alpha * (target - float(Q[s, a]))
            s = s2
            if done:
                break
    return Q

Q_lt = qlearn_looptrap()
pi = [int(np.argmax(Q_lt[s])) for s in range(5)]
pi


In [ ]:
env = LoopTrapEnv()
s = env.reset()
traj = [s]
ret = 0.0
for _ in range(25):
    a = int(np.argmax(Q_lt[s]))
    s, r, done = env.step(a)
    traj.append(s)
    ret += r
    if done:
        break
traj, ret


> Ejercicio: añade **penalización por paso** o haz que la recompensa del estado 2 decrezca con visitas.
> ¿La política sigue "hackeando" la recompensa?


In [ ]:
## Tus respuestas

#### **Ejercicios**

##### **Ejercicio 1 - Modelar un problema como MDP (formalización)**

Elige 1 de estos escenarios: (i) GridWorld con obstáculos, (ii) robot aspirador en una habitación, (iii) recomendador de noticias con "fatiga" del usuario.

1. Define $S, A, P(s'|s,a), R(s,a), \gamma$.
2. Da **dos** versiones de recompensa: una "simple" y otra "robusta".

**Entregable.** Una tabla con $S,A,R,\gamma$ + explicación breve (máx. 12 líneas).

##### **Ejercicio 2 - Bellman a mano (valor bajo una política)**

 Para un MDP pequeño (4-6 estados), fija una política $\pi$ y calcula a mano:

* $V^\pi(s)$ para cada estado resolviendo ecuaciones de Bellman.
* $Q^\pi(s,a)$ para 2 acciones por estado.

**Entregable.** Sistema de ecuaciones + solución numérica + interpretación (¿qué estados son "buenos" y por qué?).

##### **Ejercicio 3 - Evaluación de política por simulación vs. por ecuaciones**
En el mismo MDP del ejercicio 2:

1. Estima $V^\pi(s)$ por Monte Carlo (promedio de retornos).
2. Compara con tu solución exacta.

**Entregable.** Tabla "exacto vs estimado" + discusión de error y varianza.

##### **Ejercicio 4 - Q-learning tabular: exploración y convergencia**

**Enunciado.** Entrena Q-learning tabular en un entorno discreto (GridWorld/Taxi/FrozenLake).

* Prueba **3** esquemas de exploración: $\epsilon$ fijo, decaimiento lineal, decaimiento exponencial.
* Mide: retorno promedio, tasa de éxito, pasos por episodio.

**Entregable.** 3 curvas + tabla comparativa + conclusión.

##### **Ejercicio 5 - Ablación mínimo**

Con el mejor esquema del ejercicio 4, ejecuta 4 variantes:

1. sin descuento ($\gamma=1$)), 2) ($\gamma$) bajo, 3) ($\alpha$) alto, 4) ($\alpha$) bajo.

**Entregable.** Tabla de métricas final + explicación causal (por qué cambia el comportamiento).

##### **Ejercicio 6 - DQN (o aproximación con función): por qué ayuda**

Si el cuaderno incluye DQN (o aproximación con red):

1. Compara tabular vs aproximación en un entorno con estado más grande.
2. Reporta inestabilidad: oscilaciones, divergencia, sensibilidad a seeds.

**Entregable.** Curvas + 5 observaciones sobre estabilidad/fragilidad.


##### **Ejercicio 7 -REINFORCE: gradiente de política con baseline**

Implementa/usa REINFORCE en un entorno tipo CartPole.

* Variante A: sin baseline.
* Variante B: con baseline (promedio móvil o value function).

**Entregable.** Curvas comparadas + explicación de varianza (qué mejora y qué no).

##### **Ejercicio 8 - Q-learning vs Policy Gradient (cuándo conviene cada uno)**

En el mismo entorno, corre Q-learning/DQN y REINFORCE (o PPO si está). Analiza: estabilidad, sample-efficiency, sensibilidad a hiperparámetros.

**Entregable.** Una tabla "pros/cons" + evidencia (curvas/estadísticas) + recomendación.

##### **Ejercicio 9 - "Robótica" simplificada: control continuo y reward shaping**

Usa un entorno de control continuo (por ejemplo, Pendulum o equivalente disponible).

1. Diseña 2 recompensas: (A) original, (B) con shaping.
2. Evalúa si el shaping acelera el aprendizaje pero introduce sesgos.

**Entregable.** Curvas + comparación de políticas finales + reflexión: ¿qué sacrificas por aprender más rápido?

##### **Ejercicio 10 - Recomendación como bandit contextual**

Simula un recomendador:

* Contexto ($x$) (features del usuario/sesión), acciones = items, recompensa = click/tiempo.
  Implementa 2 métodos: ($\epsilon$)-greedy vs UCB o Thompson (si lo manejan).

**Entregable.** Regret acumulado + tasa de click + discusión de exploración/explotación.


##### **Ejercicio 11 - Preferencias humanas: modelo Bradley–Terry (reward model)**

Construye un dataset pequeño de preferencias (pares A/B con etiqueta "A mejor que B").

* Entrena un "reward model" simple (logístico/Bradley-Terry) que prediga preferencias.
* Evalúa accuracy y calibration (si puedes).

**Entregable.** Métricas + 5 ejemplos donde el modelo falla + hipótesis del porqué.

##### **Ejercicio 12 - RLHF simplificado: política optimizada con recompensa aprendida**

Usa tu reward model del ejercicio 11 para mejorar una política (aunque sea en un entorno "elemental" de texto/acciones discretas).

* Define una política $\pi_\theta$ y actualízala para maximizar la recompensa del reward model.

**Entregable.** Antes/después: distribución de acciones, recompensa media, ejemplos cualitativos.

##### **Ejercicio 13 - Penalización KL (estabilidad y "no desviarse" del modelo base)**

Implementa el objetivo:

$$
\max_\pi \ \mathbb{E}[r] - \beta , D_{KL}(\pi ,|, \pi_{ref})
$$

* Prueba 3 valores de $\beta$.
* Observa: "calidad" vs "drift" de la política.

**Entregable.** Curvas + tabla: recompensa vs KL + conclusión (qué $\beta$ elegirías y por qué).

##### **Ejercicio 14 - Reward hacking (demostración controlada)**

Diseña una recompensa que "parezca correcta" pero permita un atajo indeseable.
Ejemplos: maximizar "velocidad" sin penalizar choques; maximizar "clics" sin penalizar desinformación.

1. Muestra el comportamiento problemático.
2. Propón una recompensa mejor (con constraints/regularización).
 
**Entregable.** Evidencia del hack + nueva recompensa + evaluación comparativa.

##### **Ejercicio 15 - Checklist de seguridad/estabilidad para tu agente**

Crea un checklist con al menos 10 ítems para auditar un experimento de RL/RLHF:

* datos (sesgo/ruido), recompensas (especificación), estabilidad (seeds), evaluación (OOD), monitorización (drift), límites (constraints).

**Entregable.** Checklist + breve justificación por ítem.



In [ ]:
## Tus respuestas